In [1]:
import json
import numpy as np
import pandas as pd
import psycopg2

# Connect to the PostgreSQL database

First, connect to the database.

In [2]:
with open("config.json") as f:
    conf = json.load(f)

In [3]:
conn_str = "host={} dbname={} user={} password={}".format(conf["host"], conf["database"], conf["user"], conf["password"])

In [4]:
conn = psycopg2.connect(conn_str)
conn.autocommit = True # Allow the notebook to commit transactions (like creating a table) to the connected database.

# Join the tables

Query the database to join the tables containing the areas of census blocks, the populations of census blocks and the median household incomes of census block groups. We'll also convert the land area field from square meters to square miles and then calculate the population per square mile for each block.

In [5]:
pd.read_sql("""CREATE TABLE IF NOT EXISTS block_pops_incomes AS
SELECT censusblocks.geoid10 AS block_fips,
       income.statea AS state_fips,
       income.countya AS county_fips,
       income.tracta AS tract_fips,
       income.blkgrpa AS block_group_fips,
       population.state AS state,
       population.county AS county,
       population.name AS block,
       population.population,
       censusblocks.aland10 AS sq_meters,
       censusblocks.aland10 * 0.000000386102 AS sq_miles,
       (population.population / NULLIF((censusblocks.aland10 * 0.000000386102), 0)) AS pop_per_sq_mile,
       CONCAT(population.statea,
              population.countya,
              population.tracta,
              population.blkgrpa) AS block_group,
       income.median_hh_income AS block_group_income
FROM population
INNER JOIN censusblocks ON population.statea = censusblocks.statefp10
AND population.countya = censusblocks.countyfp10
AND population.tracta = censusblocks.tractce10
AND population.blocka = censusblocks.blockce10
LEFT JOIN income ON population.statea = income.statea
AND population.countya = income.countya
AND population.tracta = income.tracta
AND population.blkgrpa = income.blkgrpa;


SELECT *
FROM block_pops_incomes
LIMIT 5;""", con=conn)

,block_fips,state_fips,county_fips,tract_fips,block_group_fips,state,county,block,population,sq_meters,sq_miles,pop_per_sq_mile,block_group,block_group_income,block_group_income_quartile
0,010730001001000,01,073,000100,1,Alabama,Jefferson County,Block 1000,0,136484.0,0.052697,0.000000,010730001001,24441,1
1,010730001001001,01,073,000100,1,Alabama,Jefferson County,Block 1001,0,17127.0,0.006613,0.000000,010730001001,24441,1
2,010730001001002,01,073,000100,1,Alabama,Jefferson County,Block 1002,0,989592.0,0.382083,0.000000,010730001001,24441,1
3,010730001001003,01,073,000100,1,Alabama,Jefferson County,Block 1003,115,70938.0,0.027389,4198.719375,010730001001,24441,1
4,010730001001004,01,073,000100,1,Alabama,Jefferson County,Block 1004,111,489466.0,0.188984,587.351927,010730001001,24441,1


Query the database to join the tables containing the areas, populations and incomes of blocks and block groups with the 2012 broadband deployment data.

In [6]:
pd.read_sql("""CREATE TABLE IF NOT EXISTS deployment_pops_12 AS
SELECT deployment_2012.fullfipsid,
       block_pops_incomes.block_fips,
       block_pops_incomes.state,
       block_pops_incomes.county,
       block_pops_incomes.population,
       block_pops_incomes.sq_miles,
       block_pops_incomes.pop_per_sq_mile,
       count(DISTINCT frn) AS num_providers,
       block_pops_incomes.block_group,
       block_pops_incomes.block_group_income
FROM block_pops_incomes
INNER JOIN deployment_2012 ON block_pops_incomes.block_fips = deployment_2012.fullfipsid
WHERE deployment_2012.end_user_cat = '1'
  AND deployment_2012.provider_type = '1'
  AND (deployment_2012.maxaddown = '8'
       OR deployment_2012.maxaddown = '9'
       OR deployment_2012.maxaddown = '10'
       OR deployment_2012.maxaddown = '11')
  AND (deployment_2012.maxadup = '5'
       OR deployment_2012.maxadup = '6'
       OR deployment_2012.maxadup = '7'
       OR deployment_2012.maxadup = '8'
       OR deployment_2012.maxadup = '9'
       OR deployment_2012.maxadup = '10'
       OR deployment_2012.maxadup = '11')
  AND (deployment_2012.transtech = '10'
       OR deployment_2012.transtech = '11'
       OR deployment_2012.transtech = '20'
       OR deployment_2012.transtech = '30'
       OR deployment_2012.transtech = '40'
       OR deployment_2012.transtech = '41'
       OR deployment_2012.transtech = '50')
GROUP BY deployment_2012.fullfipsid,
         block_pops_incomes.block_fips,
         block_pops_incomes.state,
         block_pops_incomes.county,
         block_pops_incomes.population,
         block_pops_incomes.sq_miles,
         block_pops_incomes.pop_per_sq_mile,
         block_pops_incomes.block_group,
         block_pops_incomes.block_group_income;


SELECT *
FROM deployment_pops_12
LIMIT 5;""", con=conn)

,fullfipsid,block_fips,state,county,population,sq_miles,pop_per_sq_mile,num_providers,block_group,block_group_income
0,020200003001096,020200003001096,Alaska,Anchorage Municipality,0,0.333240,0.000000,1,020200003001,53788
1,020200003001139,020200003001139,Alaska,Anchorage Municipality,0,0.058716,0.000000,1,020200003001,53788
2,020200003001143,020200003001143,Alaska,Anchorage Municipality,0,0.050253,0.000000,1,020200003001,53788
3,020200003001144,020200003001144,Alaska,Anchorage Municipality,1592,0.448081,3552.925798,1,020200003001,53788
4,020200003001145,020200003001145,Alaska,Anchorage Municipality,26,0.027289,952.754263,1,020200003001,53788


Query the database to join the tables containing the areas and populations of census blocks with the 2014 broadband deployment data.

In [7]:
pd.read_sql("""CREATE TABLE IF NOT EXISTS deployment_pops_14 AS
SELECT deployment_2014.blockcode,
       block_pops_incomes.block_fips,
       block_pops_incomes.state,
       block_pops_incomes.county,
       block_pops_incomes.population,
       block_pops_incomes.sq_miles,
       block_pops_incomes.pop_per_sq_mile,
       count(DISTINCT frn) AS num_providers,
       block_pops_incomes.block_group,
       block_pops_incomes.block_group_income
FROM block_pops_incomes
INNER JOIN deployment_2014 ON block_pops_incomes.block_fips = deployment_2014.blockcode
WHERE deployment_2014.consumer = '1'
  AND deployment_2014.maxaddown >= 25
  AND deployment_2014.maxadup >= 3
  AND (deployment_2014.techcode = '10'
       OR deployment_2014.techcode = '11'
       OR deployment_2014.techcode = '12'
       OR deployment_2014.techcode = '20'
       OR deployment_2014.techcode = '30'
       OR deployment_2014.techcode = '40'
       OR deployment_2014.techcode = '41'
       OR deployment_2014.techcode = '42'
       OR deployment_2014.techcode = '43'
       OR deployment_2014.techcode = '50')
GROUP BY deployment_2014.blockcode,
         block_pops_incomes.block_fips,
         block_pops_incomes.state,
         block_pops_incomes.county,
         block_pops_incomes.population,
         block_pops_incomes.sq_miles,
         block_pops_incomes.pop_per_sq_mile,
         block_pops_incomes.block_group,
         block_pops_incomes.block_group_income;


SELECT *
FROM deployment_pops_14
LIMIT 5;""", con=conn)

,blockcode,block_fips,state,county,population,sq_miles,pop_per_sq_mile,num_providers,block_group,block_group_income
0,010010201001000,010010201001000,Alabama,Autauga County,61,0.186344,327.352204,2,010010201001,None
1,010010201001001,010010201001001,Alabama,Autauga County,0,0.000186,0.000000,1,010010201001,None
2,010010201001002,010010201001002,Alabama,Autauga County,0,0.001349,0.000000,1,010010201001,None
3,010010201001003,010010201001003,Alabama,Autauga County,75,0.197615,379.526373,1,010010201001,None
4,010010201001004,010010201001004,Alabama,Autauga County,0,0.000289,0.000000,1,010010201001,None


Query the database to join the tables containing the areas and populations of census blocks with the 2016 broadband deployment data.

In [8]:
pd.read_sql("""CREATE TABLE IF NOT EXISTS deployment_pops_16 AS
SELECT deployment_2016.blockcode,
       block_pops_incomes.block_fips,
       block_pops_incomes.state,
       block_pops_incomes.county,
       block_pops_incomes.population,
       block_pops_incomes.sq_miles,
       block_pops_incomes.pop_per_sq_mile,
       count(DISTINCT frn) AS num_providers,
       block_pops_incomes.block_group,
       block_pops_incomes.block_group_income
FROM block_pops_incomes
INNER JOIN deployment_2016 ON block_pops_incomes.block_fips = deployment_2016.blockcode
WHERE deployment_2016.consumer = '1'
  AND deployment_2016.maxaddown >= 25
  AND deployment_2016.maxadup >= 3
  AND (deployment_2016.techcode = '10'
       OR deployment_2016.techcode = '11'
       OR deployment_2016.techcode = '12'
       OR deployment_2016.techcode = '20'
       OR deployment_2016.techcode = '30'
       OR deployment_2016.techcode = '40'
       OR deployment_2016.techcode = '41'
       OR deployment_2016.techcode = '42'
       OR deployment_2016.techcode = '43'
       OR deployment_2016.techcode = '50')
GROUP BY deployment_2016.blockcode,
         block_pops_incomes.block_fips,
         block_pops_incomes.state,
         block_pops_incomes.county,
         block_pops_incomes.population,
         block_pops_incomes.sq_miles,
         block_pops_incomes.pop_per_sq_mile,
         block_pops_incomes.block_group,
         block_pops_incomes.block_group_income;


SELECT *
FROM deployment_pops_16
LIMIT 5;""", con=conn)

,blockcode,block_fips,state,county,population,sq_miles,pop_per_sq_mile,num_providers,block_group,block_group_income
0,010010201001000,010010201001000,Alabama,Autauga County,61,0.186344,327.352204,2,010010201001,None
1,010010201001001,010010201001001,Alabama,Autauga County,0,0.000186,0.000000,1,010010201001,None
2,010010201001002,010010201001002,Alabama,Autauga County,0,0.001349,0.000000,1,010010201001,None
3,010010201001003,010010201001003,Alabama,Autauga County,75,0.197615,379.526373,1,010010201001,None
4,010010201001004,010010201001004,Alabama,Autauga County,0,0.000289,0.000000,1,010010201001,None


Query the database to join the 2012 and 2014 deployment, population and income tables.

In [9]:
pd.read_sql("""CREATE TABLE IF NOT EXISTS deployment_pops_12_14 as
SELECT deployment_pops_14.blockcode,
       deployment_pops_14.state,
       deployment_pops_14.county,
       deployment_pops_14.population,
       deployment_pops_14.sq_miles,
       deployment_pops_14.pop_per_sq_mile,
       deployment_pops_14.block_group,
       deployment_pops_14.block_group_income,
       deployment_pops_14.num_providers AS num_providers_2014,
       deployment_pops_12.num_providers AS num_providers_2012
FROM deployment_pops_14
LEFT JOIN deployment_pops_12 ON deployment_pops_14.blockcode = deployment_pops_12.fullfipsid;


SELECT *
FROM deployment_pops_12_14
LIMIT 5;""", con=conn)

,blockcode,state,county,population,sq_miles,pop_per_sq_mile,block_group,block_group_income,num_providers_2014,num_providers_2012
0,010010201001002,Alabama,Autauga County,0,0.001349,0.000000,010010201001,NaN,1,None
1,010010201002003,Alabama,Autauga County,0,0.045992,0.000000,010010201002,77813.0,1,None
2,010010201002030,Alabama,Autauga County,100,0.016882,5923.630981,010010201002,77813.0,2,None
3,010010202001012,Alabama,Autauga County,248,0.109535,2264.112216,010010202001,25179.0,2,None
4,010010202002010,Alabama,Autauga County,9,0.003643,2470.316084,010010202002,45104.0,2,None


Query the database to join the 2014 and 2016 deployment, population and income tables.

In [10]:
pd.read_sql("""CREATE TABLE IF NOT EXISTS deployment_pops_14_16 as
SELECT deployment_pops_16.blockcode,
       deployment_pops_16.state,
       deployment_pops_16.county,
       deployment_pops_16.population,
       deployment_pops_16.sq_miles,
       deployment_pops_16.pop_per_sq_mile,
       deployment_pops_16.block_group,
       deployment_pops_16.block_group_income,
       deployment_pops_16.num_providers AS num_providers_2016,
       deployment_pops_14.num_providers AS num_providers_2014
FROM deployment_pops_16
LEFT JOIN deployment_pops_14 ON deployment_pops_16.blockcode = deployment_pops_14.blockcode;


SELECT *
FROM deployment_pops_14_16
LIMIT 5;""", con=conn)

,blockcode,state,county,population,sq_miles,pop_per_sq_mile,block_group,block_group_income,num_providers_2016,num_providers_2014
0,010010201002003,Alabama,Autauga County,0,0.045992,0.000000,010010201002,77813,1,1
1,010010203001006,Alabama,Autauga County,69,0.013661,5050.994969,010010203001,55222,2,2
2,010010204002022,Alabama,Autauga County,46,0.013075,3518.071814,010010204002,45962,2,2
3,010010208011056,Alabama,Autauga County,4,0.072149,55.440565,010010208011,70625,2,2
4,010010209004010,Alabama,Autauga County,0,0.032962,0.000000,010010209004,57228,1,1


# Analyze

How many blocks that had at least one broadband service provider in 2014 did not have any in 2012?

In [11]:
deployment_pops_12_14 = pd.read_sql("""SELECT *
FROM deployment_pops_12_14;""", con=conn)
deployment_pops_12_14.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5588651 entries, 0 to 5588650
Data columns (total 10 columns):
blockcode             object
state                 object
county                object
population            int64
sq_miles              float64
pop_per_sq_mile       float64
block_group           object
block_group_income    float64
num_providers_2014    int64
num_providers_2012    float64
dtypes: float64(4), int64(2), object(4)
memory usage: 426.4+ MB


In [12]:
deployment_pops_12_14.head()

,blockcode,state,county,population,sq_miles,pop_per_sq_mile,block_group,block_group_income,num_providers_2014,num_providers_2012
0,010010201001002,Alabama,Autauga County,0,0.001349,0.000000,010010201001,NaN,1,NaN
1,010010201002003,Alabama,Autauga County,0,0.045992,0.000000,010010201002,77813.0,1,NaN
2,010010201002030,Alabama,Autauga County,100,0.016882,5923.630981,010010201002,77813.0,2,NaN
3,010010202001012,Alabama,Autauga County,248,0.109535,2264.112216,010010202001,25179.0,2,NaN
4,010010202002010,Alabama,Autauga County,9,0.003643,2470.316084,010010202002,45104.0,2,NaN


In [13]:
deployment_pops_12_14[(deployment_pops_12_14["num_providers_2014"] >= 1) & (deployment_pops_12_14["num_providers_2012"].isnull())].head()

,blockcode,state,county,population,sq_miles,pop_per_sq_mile,block_group,block_group_income,num_providers_2014,num_providers_2012
0,010010201001002,Alabama,Autauga County,0,0.001349,0.000000,010010201001,NaN,1,NaN
1,010010201002003,Alabama,Autauga County,0,0.045992,0.000000,010010201002,77813.0,1,NaN
2,010010201002030,Alabama,Autauga County,100,0.016882,5923.630981,010010201002,77813.0,2,NaN
3,010010202001012,Alabama,Autauga County,248,0.109535,2264.112216,010010202001,25179.0,2,NaN
4,010010202002010,Alabama,Autauga County,9,0.003643,2470.316084,010010202002,45104.0,2,NaN


In [14]:
deployment_pops_12_14[(deployment_pops_12_14["num_providers_2014"] >= 1) & (deployment_pops_12_14["num_providers_2012"].isnull())].count()

blockcode             3994519
state                 3994519
county                3994519
population            3994519
sq_miles              3994519
pop_per_sq_mile       3913227
block_group           3994519
block_group_income    3915624
num_providers_2014    3994519
num_providers_2012          0
dtype: int64

OK. So, 3,994,519 blocks — 71.5 percent of all blocks that had at least one broadband service provider in 2014 — did not have any in 2012.

How many blocks that had at least one broadband service provider in 2016 did not have any in 2014?

In [15]:
deployment_pops_14_16 = pd.read_sql("""SELECT *
FROM deployment_pops_14_16;""", con=conn)
deployment_pops_14_16.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6001805 entries, 0 to 6001804
Data columns (total 10 columns):
blockcode             object
state                 object
county                object
population            int64
sq_miles              float64
pop_per_sq_mile       float64
block_group           object
block_group_income    float64
num_providers_2016    int64
num_providers_2014    float64
dtypes: float64(4), int64(2), object(4)
memory usage: 457.9+ MB


In [16]:
deployment_pops_14_16.head()

,blockcode,state,county,population,sq_miles,pop_per_sq_mile,block_group,block_group_income,num_providers_2016,num_providers_2014
0,010010201002003,Alabama,Autauga County,0,0.045992,0.000000,010010201002,77813.0,1,1.0
1,010010203001006,Alabama,Autauga County,69,0.013661,5050.994969,010010203001,55222.0,2,2.0
2,010010204002022,Alabama,Autauga County,46,0.013075,3518.071814,010010204002,45962.0,2,2.0
3,010010208011056,Alabama,Autauga County,4,0.072149,55.440565,010010208011,70625.0,2,2.0
4,010010209004010,Alabama,Autauga County,0,0.032962,0.000000,010010209004,57228.0,1,1.0


In [17]:
deployment_pops_14_16[(deployment_pops_14_16["num_providers_2016"] >= 1) & (deployment_pops_14_16["num_providers_2014"].isnull())].head()

,blockcode,state,county,population,sq_miles,pop_per_sq_mile,block_group,block_group_income,num_providers_2016,num_providers_2014
5,010030101002100,Alabama,Baldwin County,46,0.840305,54.742050,010030101002,49115.0,1,NaN
32,010030115011025,Alabama,Baldwin County,118,0.036480,3234.608214,010030115011,55247.0,1,NaN
35,010070100021128,Alabama,Bibb County,6,0.001454,4126.376804,010070100021,39125.0,1,NaN
40,010090502002102,Alabama,Blount County,15,0.214578,69.904738,010090502002,49167.0,1,NaN
46,010139527002049,Alabama,Butler County,0,0.002227,0.000000,010139527002,48542.0,1,NaN


In [18]:
deployment_pops_14_16[(deployment_pops_14_16["num_providers_2016"] >= 1) & (deployment_pops_14_16["num_providers_2014"].isnull())].count()

blockcode             674441
state                 674441
county                674441
population            674441
sq_miles              674441
pop_per_sq_mile       651343
block_group           674441
block_group_income    662969
num_providers_2016    674441
num_providers_2014         0
dtype: int64

OK. So, 674,441 blocks — 11.2 percent of all blocks that had at least one broadband service provider in 2016 — did not have any in 2014.

What are the mean and median numbers of broadband service providers in each block in 2012, 2014 and 2016?

In [19]:
pd.read_sql("""SELECT avg(num_providers) AS avg_num_providers
FROM deployment_pops_12;""", con=conn)

,avg_num_providers
0,1.173534


In [20]:
pd.read_sql("""SELECT PERCENTILE_CONT(0.5) WITHIN GROUP(
                                         ORDER BY num_providers) AS median_num_providers
FROM deployment_pops_12;""", con=conn)

,median_num_providers
0,1.0


In [21]:
pd.read_sql("""SELECT avg(num_providers) AS avg_num_providers
FROM deployment_pops_14;""", con=conn)

,avg_num_providers
0,1.254612


In [22]:
pd.read_sql("""SELECT PERCENTILE_CONT(0.5) WITHIN GROUP(
                                         ORDER BY num_providers) AS median_num_providers
FROM deployment_pops_14;""", con=conn)

,median_num_providers
0,1.0


In [23]:
pd.read_sql("""SELECT avg(num_providers) AS avg_num_providers
FROM deployment_pops_16;""", con=conn)

,avg_num_providers
0,1.383835


In [24]:
pd.read_sql("""SELECT PERCENTILE_CONT(0.5) WITHIN GROUP(
                                         ORDER BY num_providers) AS median_num_providers
FROM deployment_pops_16;""", con=conn)

,median_num_providers
0,1.0


OK. So, in 2012, the mean and median numbers of broadband service providers in each block was 1.17 and one, respectively.

In 2014, those figures were 1.26 and one.

And in 2016, those figures were 1.38 and one.

What were the mean and median population densities of the blocks that had at least one broadband service provider in 2014 and none in 2012?

In [25]:
pd.read_sql("""SELECT avg(pop_per_sq_mile) AS avg_pop_per_sq_mile
FROM deployment_pops_12_14
WHERE num_providers_2012 IS NULL;""", con=conn)

,avg_pop_per_sq_mile
0,4467.025208


In [26]:
pd.read_sql("""SELECT PERCENTILE_CONT(0.5) WITHIN GROUP(
                                         ORDER BY pop_per_sq_mile) AS median_pop_per_sq_mile
FROM deployment_pops_12_14
WHERE num_providers_2012 IS NULL;""", con=conn)

,median_pop_per_sq_mile
0,1725.039694


 What were the mean and median population densities of the blocks that had at least one broadband service provider in 2016 and none in 2014?

In [27]:
pd.read_sql("""SELECT avg(pop_per_sq_mile) AS avg_pop_per_sq_mile
FROM deployment_pops_14_16
WHERE num_providers_2014 IS NULL;""", con=conn)

,avg_pop_per_sq_mile
0,1592.54482


In [28]:
pd.read_sql("""SELECT PERCENTILE_CONT(0.5) WITHIN GROUP(
                                         ORDER BY pop_per_sq_mile) AS median_pop_per_sq_mile
FROM deployment_pops_14_16
WHERE num_providers_2014 IS NULL;""", con=conn)

,median_pop_per_sq_mile
0,15.102389


OK. So, the mean and median population densities of the blocks that had at least one broadband service provider in 2014 and none in 2012 were 4,467 people per square mile and 1,725 people per square mile, respectively.

And the mean and median population densities of the blocks that had at least one broadband service provider in 2016 and none in 2014 were 1,593 people per square mile and 15 people per square mile, respectively.

What is the distribution of the population among income quartiles of blocks that had at least one broadband service provider in 2014 and none in 2012?

In [29]:
pd.read_sql("""WITH temp_table AS
  (SELECT population,
          num_providers_2012,
          block_group_income,
          ntile(4) OVER (
                         ORDER BY block_group_income) AS quartile
   FROM deployment_pops_12_14)
SELECT sum(population) AS sum_population,
       quartile,
       avg(block_group_income) AS average_block_group_income
FROM temp_table
WHERE num_providers_2012 IS NULL
GROUP BY quartile
ORDER BY quartile DESC;""", con=conn)

,sum_population,quartile,average_block_group_income
0,65379033,4,103684.661179
1,46428901,3,62278.175101
2,40199591,2,45997.706258
3,42995234,1,28801.835881


What is the distribution of the population among income quartiles of blocks that had at least one broadband service provider in 2016 and none in 2014?

In [30]:
pd.read_sql("""WITH temp_table AS
  (SELECT population,
          num_providers_2014,
          block_group_income,
          ntile(4) OVER (
                         ORDER BY block_group_income) AS quartile
   FROM deployment_pops_14_16)
SELECT sum(population) AS sum_population,
       quartile,
       avg(block_group_income) AS average_block_group_income
FROM temp_table
WHERE num_providers_2014 IS NULL
GROUP BY quartile
ORDER BY quartile DESC;""", con=conn)

,sum_population,quartile,average_block_group_income
0,1549645,4,95037.007871
1,2322288,3,60760.548060
2,2797297,2,45743.813032
3,2848711,1,29742.028937
